In [ ]:
import pickle
import numpy as np 
import pandas as pd
# import matplotlib.pyplot as plt
import pickle

from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import Embedding, Dense, TimeDistributed, Concatenate, BatchNormalization
from keras.layers import Bidirectional, Activation, Dropout, CuDNNGRU, Conv1D

from sklearn.model_selection import train_test_split, KFold
from keras.callbacks import EarlyStopping
from keras.metrics import categorical_accuracy
from keras import backend as K
from keras.regularizers import l1, l2
import tensorflow as tf

In [ ]:
from lmcodes.getdata import get_data

In [ ]:
train_path="cullpdb+profile_6133_filtered.npy"
test_path="cb513+profile_split1.npy"
maxlen_seq=700 # protein residues padded to 700
r = 700
f = 57

cb6133filtered = np.load(train_path)
cb513          = np.load(test_path)

cb6133filtered = cb6133filtered[:100]
cb513          = np.copy(cb6133filtered)

In [ ]:
x_train_lm, y_train_lm, x_test_lm, y_test_lm, tokenizer_encoder, tokenizer_decoder, n_words, n_tags = \
    get_data(cb6133filtered, cb513, lm_pretrain=True)

In [ ]:
# print(x_train[1][0])
print(len(tokenizer_encoder.word_index), len(tokenizer_decoder.word_index))

In [ ]:
from lmcodes.createmodel import CNN_BIGRU, decode_results, run_test, model_train

In [ ]:
import datetime
salt = "-".join(str(datetime.datetime.now()).split()) + "-lm-"

# Change the validation data from here


history, lm_model = model_train(
    x_train_lm, y_train_lm, maxlen_seq, n_words, n_tags, 
    x_val=x_train_lm, y_val=y_train_lm, n_epoch=1, batch_size=10,
    mpath="savedModels/salt-" + salt + "-")

In [ ]:
y_pred = lm_model.predict(x_train_lm)

In [ ]:
np.argmax(y_pred[0], -1)

In [ ]:
np.argmax(y_train[0], -1)

In [ ]:
x_train[0].shape, x_train[1].shape, x_train[2].shape

In [ ]:
import os
files = os.listdir("savedModels")
bestweights = ""
for f in sorted(files):
    if salt in f:
        print(f)
        bestweights = f
print(bestweights)
lm_model.load_weights("savedModels/" + bestweights)
lm_model.evaluate(x_train_lm, y_train_lm)

In [ ]:
import datetime
salt = "-".join(str(datetime.datetime.now()).split()) + "-cf-"

x_train, y_train, x_test, y_test, tokenizer_encoder_, tokenizer_decoder_, n_words, n_tags = \
    get_data(cb6133filtered, cb513, lm_pretrain=False, tokenizer_encoder=tokenizer_encoder)

history, cfmodel = model_train(
    x_train, y_train, maxlen_seq, n_words, n_tags, 
    x_val=x_train, y_val=y_train, n_epoch=1, batch_size=10,
    mpath="savedModels/salt-" + salt + "-", lmmodel=lm_model)

In [ ]:
print(model2.layers[6].get_weights()[0][0][0])
print(model.layers[6].get_weights()[0][0][0])
print(model.optimizer.iterations)

In [ ]:
from keras.metrics import *
model2.compile(
        optimizer="nadam",
        loss = "categorical_crossentropy",
        metrics = ["accuracy", accuracy])
print(model2.optimizer.iterations)

In [ ]:
model2.fit(x_train, y_train, epochs=10, batch_size=10)

In [ ]:
model.evaluate(x=x_train, y=y_train)
model.load_weights("./savedModels/weights-improvement-03-0.46.hdf5")
model.evaluate(x=x_train, y=y_train)

In [ ]:
import seaborn as sns
sns.distplot(x_train[-1][x_train[-1]>0.1])
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(x_train[-1].reshape(100, -1))
print(pca.explained_variance_ratio_)
print(pca.singular_values_)